In [1]:
#enveloping potential
# simple Example plot Enveloped Potential with two Harmonic Oscilators
##Imports:
import os, sys as csys, datetime, tempfile
import math, numpy as np, pandas as pd
import matplotlib.pyplot as plt

from tqdm.notebook import tqdm


csys.path.append(os.getcwd()+"/../../..")

#Ensembler
import ensembler.potentials.TwoD as pot2D

from ensembler.samplers.stochastic import metropolisMonteCarloIntegrator
from ensembler.conditions.box_conditions import periodicBoundaryCondition
from ensembler.system.basic_system import system

import ensembler.visualisation.plotPotentials as exPlot

plot_resolution= 1200 # lowRes but fast: 120
%matplotlib inline

In [2]:
#ENERGIES Sampling - CURRENTLY NOT USED!
def write_out_etraj(traj, out_path, V1, V2):
    visited_positions =  np.array(list(map(np.array, traj.position)))
    state1 = V1.ene(visited_positions)
    state2 = V2.ene(visited_positions)
    Vrenergies = np.array(traj.total_potential_energy)

    out_file = open(out_path, "w")
    out_file.write("t\tV1\tV2\tVr\n")#HEADER
    for t, V1, V2, Vr in zip(traj.index, state1, state2, Vrenergies):
        out_file.write(str(t)+"\t"+str(V1)+"\t"+str(V2)+"\t"+str(Vr)+"\n")

    out_file.close()

In [3]:
sys = None
def simulate_replicas(out_dir:str, barrier, opt_s, replicas=10, simulation_steps=1000000):
    if(not os.path.exists(out_dir)):
        os.mkdir(out_dir)
    
    start_t = datetime.datetime.now()
    #Potentials
    amplitude = barrier/2
    shift  = 90
    Eoff = [0,0]
    simulation_space =((-180,180), (-180,180))

    V1 = pot2D.wavePotential(phase_shift=(shift,shift), multiplicity=(2.0, 2.0), amplitude=(amplitude, amplitude), radians=False)
    V2 = pot2D.wavePotential(phase_shift=(0.0, 0.0), multiplicity=(2.0, 2.0), amplitude=(amplitude, amplitude), radians=False)
    edsPot = pot2D.envelopedPotential(V_is=[V1, V2], s=1.0, eoff=Eoff)

    #exPlot.plot_2D_2State_EDS_potential(eds_pot=edsPot, out_path = out_dir+"/SimSpace_"+str(barrier)+"kT.png", point_resolution=plot_resolution)

    #Simulation Setup
    svals = sorted([10, 1, 0.5, 0.25, 0.1, 0.05, 0.025, 0.01]+[opt_s], reverse=True)
    ##Build modules
    periodic_bound = periodicBoundaryCondition(boundary=simulation_space)
    integrator = metropolisMonteCarloIntegrator(fixed_step_size=[1,1], max_iteration_tillAccept=1000, randomness_increase_factor=1) 
    
    ###Build System
    sys=system(potential=edsPot, sampler=integrator, conditions=[periodic_bound])
    print(sys.potential.V_is[0])
    print("radians: ", sys.potential.V_is[0].radians)
    print(sys.nDimensions, edsPot.constants[edsPot.nDimensions])
    print("svalue sequence - #s="+str(len(svals))+" ", svals)
    
    ## SIMULATE
    start=0
    syst = None
    for replica in tqdm(range(start, replicas), desc="Replicas: ", leave=False):
        ##INIT REPLICA
        ###fILE hANDLING
        replica_out = out_dir+"/replica_"+str(replica)
        if(not os.path.exists(replica_out)):
            os.mkdir(replica_out)

        ## SIMULATE REPLICA
        start_sim = datetime.datetime.now()
        s_val_posDict = {}  #OUT
        for s in tqdm(svals, desc="Current Simulation of replica "+str(replica)+": ", leave=False):
            sys.potential.s = s
            cur_state = sys.simulate(simulation_steps, withdraw_traj=True, init_system=True)

            s_val_posDict.update({s:sys.trajectory})
            traj = sys.trajectory

            exPlot.plot_2D_2State_EDS_potential(edsPot, out_path = replica_out+"/SimSpace_withTraj_"+str(s)+"_"+str(barrier)+"kT.png", 
                                                traj=traj, point_resolution=plot_resolution, space_range=simulation_space)

            
        end_sim = datetime.datetime.now()
        duration_sim = end_sim-start_sim

        traj = sys.trajectory

        #plotting:
        print("plotting")
        start_plot = datetime.datetime.now()
        
        if(replica == 0):
            fig = exPlot.plot_2D_2State_EDS_potential_sDependency(sVal_traj_Dict=s_val_posDict, eds_pot=edsPot, space_range=simulation_space,
                                                                  plot_trajs=False, out_path=out_dir+"/s_dependent_sampling_relBarrier_"+str(barrier)+"kT.png",
                                                                  point_resolution=plot_resolution),
            
        fig = exPlot.plot_2D_2State_EDS_potential_sDependency(sVal_traj_Dict=s_val_posDict, eds_pot=edsPot, plot_trajs=True, 
                                                              out_path=replica_out+"/s_dependent_sampling_relBarrier_"+str(barrier)+"kT_withTraj.png", 
                                                              point_resolution=plot_resolution, space_range=simulation_space)
        end_plot = datetime.datetime.now()
        duration_plot = end_plot-start_plot
        
        for s in s_val_posDict:
            write_out_etraj(traj=s_val_posDict[s], out_path=replica_out+"/replica_traj_s"+str(s)+".dat", V1=sys.potential.V_is[0], V2=sys.potential.V_is[1])
        sys.write_trajectory(replica_out+"/total_replica_traj.dat")
        

    del edsPot, sys, traj 
    
    end_t = datetime.datetime.now()
    duration = end_t-start_t
    print("Done - duration: ", duration)
    print("Done - simulation duration: ", duration_sim)
    print("Done - plotting duration: ", duration_plot)


In [4]:
#run multiple replicas
tmp_dir = tempfile.gettempdir()+"/edsSim"
if(not os.path.exists(tmp_dir)):
    os.mkdir(tmp_dir)
os.chdir(tmp_dir)

print(tmp_dir)


#BUILD Potential:
#params:
replicas = 1#0

##STEPS
each_sim = 100000#0 #1000 000 #each s value and each replica

#s_est = 0.609/Delta U_barrier
optimal_ses = {2.5: 0.122,
               5: 0.061,
               10: 0.03,
               20: 0.03, #0.015
               50: 0.006}


for barrier in tqdm(optimal_ses, desc="Barrier approaches: "):
    approach_dir = tmp_dir+"/independent_simulations_with_"+str(barrier)+"kT_barriers"
    simulate_replicas(out_dir=approach_dir, barrier=barrier, opt_s=optimal_ses[barrier], replicas=replicas, simulation_steps=each_sim)



C:\Users\benja\AppData\Local\Temp/edsSim


wavePotential
	States: 1
	Dimensions: 2

	Functional:
 		V:	Sum(Matrix([
[amp_0*cos(mult_0*(phase_0 + r_0)) + yOff_0],
[amp_1*cos(mult_1*(phase_1 + r_1)) + yOff_1]])[i, 0], (i, 0, nDimensions - 1))
		dVdpos:	Matrix([[Sum(Matrix([
[-amp_0*mult_0*sin(mult_0*(phase_0 + r_0))],
[                                        0]])[i, 0], (i, 0, nDimensions - 1))], [Sum(Matrix([
[                                        0],
[-amp_1*mult_1*sin(mult_1*(phase_1 + r_1))]])[i, 0], (i, 0, nDimensions - 1))]])

	Simplified Function
		V:	Sum(Matrix([
[1.25*cos(2.0*r_0 + 3.14159265358979)],
[1.25*cos(2.0*r_1 + 3.14159265358979)]])[i, 0], (i, 0, 1))
		dVdpos:	Matrix([[Sum(Matrix([
[-2.5*sin(2.0*r_0 + 3.14159265358979)],
[                                   0]])[i, 0], (i, 0, 1))], [Sum(Matrix([
[                                   0],
[-2.5*sin(2.0*r_1 + 3.14159265358979)]])[i, 0], (i, 0, 1))]])

	Constants: 
		amp_0: 			1.25
		amp_1: 			1.25
		yOff_0: 			0
		yOff_1: 			0
		mult_0: 			2.0
		mult_1: 			2.0
		pha

Simulation:  Simulation: 100%|██████████| 100000/100000 [36:32<00:00, 45.61it/s]


plotting
Done - duration:  4:52:45.928419
Done - simulation duration:  4:51:29.483127
Done - plotting duration:  0:00:49.887139
wavePotential
	States: 1
	Dimensions: 2

	Functional:
 		V:	Sum(Matrix([
[amp_0*cos(mult_0*(phase_0 + r_0)) + yOff_0],
[amp_1*cos(mult_1*(phase_1 + r_1)) + yOff_1]])[i, 0], (i, 0, nDimensions - 1))
		dVdpos:	Matrix([[Sum(Matrix([
[-amp_0*mult_0*sin(mult_0*(phase_0 + r_0))],
[                                        0]])[i, 0], (i, 0, nDimensions - 1))], [Sum(Matrix([
[                                        0],
[-amp_1*mult_1*sin(mult_1*(phase_1 + r_1))]])[i, 0], (i, 0, nDimensions - 1))]])

	Simplified Function
		V:	Sum(Matrix([
[2.5*cos(2.0*r_0 + 3.14159265358979)],
[2.5*cos(2.0*r_1 + 3.14159265358979)]])[i, 0], (i, 0, 1))
		dVdpos:	Matrix([[Sum(Matrix([
[-5.0*sin(2.0*r_0 + 3.14159265358979)],
[                                   0]])[i, 0], (i, 0, 1))], [Sum(Matrix([
[                                   0],
[-5.0*sin(2.0*r_1 + 3.14159265358979)]])[i, 0], (i, 0

Simulation:  Simulation: 100%|██████████| 100000/100000 [1:11:15<00:00, 23.39it/s]  


...

plotting
Done - duration:  8:59:14.981567
Done - simulation duration:  8:57:38.927944
Done - plotting duration:  0:01:02.938096
wavePotential
	States: 1
	Dimensions: 2

	Functional:
 		V:	Sum(Matrix([
[amp_0*cos(mult_0*(phase_0 + r_0)) + yOff_0],
[amp_1*cos(mult_1*(phase_1 + r_1)) + yOff_1]])[i, 0], (i, 0, nDimensions - 1))
		dVdpos:	Matrix([[Sum(Matrix([
[-amp_0*mult_0*sin(mult_0*(phase_0 + r_0))],
[                                        0]])[i, 0], (i, 0, nDimensions - 1))], [Sum(Matrix([
[                                        0],
[-amp_1*mult_1*sin(mult_1*(phase_1 + r_1))]])[i, 0], (i, 0, nDimensions - 1))]])

	Simplified Function
		V:	Sum(Matrix([
[10.0*cos(2.0*r_0 + 3.14159265358979)],
[10.0*cos(2.0*r_1 + 3.14159265358979)]])[i, 0], (i, 0, 1))
		dVdpos:	Matrix([[Sum(Matrix([
[-20.0*sin(2.0*r_0 + 3.14159265358979)],
[                                    0]])[i, 0], (i, 0, 1))], [Sum(Matrix([
[                                    0],
[-20.0*sin(2.0*r_1 + 3.14159265358979)]])[i, 0],

Simulation:  Simulation: 100%|██████████| 100000/100000 [41:09<00:00, 40.49it/s]


plotting
Done - duration:  18:56:16.463897
Done - simulation duration:  18:54:34.370596
Done - plotting duration:  0:01:09.598716
wavePotential
	States: 1
	Dimensions: 2

	Functional:
 		V:	Sum(Matrix([
[amp_0*cos(mult_0*(phase_0 + r_0)) + yOff_0],
[amp_1*cos(mult_1*(phase_1 + r_1)) + yOff_1]])[i, 0], (i, 0, nDimensions - 1))
		dVdpos:	Matrix([[Sum(Matrix([
[-amp_0*mult_0*sin(mult_0*(phase_0 + r_0))],
[                                        0]])[i, 0], (i, 0, nDimensions - 1))], [Sum(Matrix([
[                                        0],
[-amp_1*mult_1*sin(mult_1*(phase_1 + r_1))]])[i, 0], (i, 0, nDimensions - 1))]])

	Simplified Function
		V:	Sum(Matrix([
[25.0*cos(2.0*r_0 + 3.14159265358979)],
[25.0*cos(2.0*r_1 + 3.14159265358979)]])[i, 0], (i, 0, 1))
		dVdpos:	Matrix([[Sum(Matrix([
[-50.0*sin(2.0*r_0 + 3.14159265358979)],
[                                    0]])[i, 0], (i, 0, 1))], [Sum(Matrix([
[                                    0],
[-50.0*sin(2.0*r_1 + 3.14159265358979)]])[i, 0

Simulation:  Simulation: 100%|██████████| 100000/100000 [24:35<00:00, 67.75it/s]


plotting
Done - duration:  7:30:48.376018
Done - simulation duration:  7:29:55.893719
Done - plotting duration:  0:00:36.121484

